In [103]:
from abc import abstractmethod

from torchtune.data import ChatFormat, InstructTemplate, AlpacaInstructTemplate
from torchtune.datasets import alpaca_dataset, alpaca_cleaned_dataset, chat_dataset, instruct_dataset, InstructDataset
from torchtune.models.llama3 import llama3_tokenizer
from datasets import Dataset, load_dataset
from torchtune.data import Message

In [82]:
dataset = load_dataset("beomi/KoAlpaca-v1.1a", split='train')

In [108]:
tmp = InstructDataset(
    tokenizer=llama3_tokenizer,
    source="beomi/KoAlpaca-v1.1a",
    template=AlpacaInstructTemplate,
    train_on_input=True,
    max_seq_len=2048,
    split="train",
    )